In [ ]:
!apt-get update
from google.colab import drive

# Use the 'force_remount' parameter to refresh credentials
drive.mount('/drive/', force_remount=True)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:2 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,544 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,181 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,998 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,263 kB]
Get:14 http://security.ubuntu.com/ubuntu jammy

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, DenseNet121
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
def cbam_block(input_tensor, ratio=8):
    # Channel Attention Module
    channel = input_tensor.shape[-1]
    shared_layer_one = layers.Dense(channel//ratio,
                                    activation='relu',
                                    kernel_initializer='he_normal',
                                    use_bias=True,
                                    bias_initializer='zeros')
    shared_layer_two = layers.Dense(channel,
                                    kernel_initializer='he_normal',
                                    use_bias=True,
                                    bias_initializer='zeros')

    avg_pool = layers.GlobalAveragePooling2D()(input_tensor)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = layers.GlobalMaxPooling2D()(input_tensor)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)

    cbam_feature = layers.Multiply()([input_tensor, cbam_feature])

    # Spatial Attention Module
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=3, keepdims=True))(cbam_feature)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=3, keepdims=True))(cbam_feature)
    concat = layers.Concatenate(axis=3)([avg_pool, max_pool])
    cbam_feature = layers.Conv2D(filters=1,
                                 kernel_size=7,
                                 strides=1,
                                 padding='same',
                                 kernel_initializer='he_normal',
                                 use_bias=False)(concat)
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)

    cbam_feature = layers.Multiply()([cbam_feature, input_tensor])
    return cbam_feature

In [ ]:
def HybridModel(input_shape, num_classes):
    # ResNet50 Model
    input_resnet = layers.Input(shape=input_shape, name='input_resnet')
    resnet50 = ResNet50(weights='imagenet', include_top=False, input_tensor=input_resnet)
    resnet50.trainable = False

    # Prefix ResNet50 layer names to avoid conflicts
    for layer in resnet50.layers:
        layer._name = 'resnet_' + layer.name

    resnet_output = resnet50.output
    resnet_output = cbam_block(resnet_output)

    # DenseNet121 Model
    input_densenet = layers.Input(shape=input_shape, name='input_densenet')
    densenet121 = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_densenet)
    densenet121.trainable = False

    # Prefix DenseNet121 layer names to avoid conflicts
    for layer in densenet121.layers:
        layer._name = 'densenet_' + layer.name

    densenet_output = densenet121.output
    densenet_output = cbam_block(densenet_output)

    # Combine Features
    combined = layers.Concatenate()([resnet_output, densenet_output])
    combined = layers.GlobalAveragePooling2D()(combined)
    combined = layers.Dense(1024, activation='relu')(combined)
    combined = layers.Dropout(0.5)(combined)
    output = layers.Dense(1, activation='sigmoid')(combined)

    model = models.Model(inputs=[input_resnet, input_densenet], outputs=output)
    return model

In [ ]:
input_shape = (32, 32, 3)
num_classes = 1

model = HybridModel(input_shape, num_classes)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, DenseNet121

# Görselleri yükleme ve ön işleme
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/drive/MyDrive/Colab Notebooks/cifake/train',
    target_size=(32, 32),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    '/drive/MyDrive/Colab Notebooks/cifake/test',
    target_size=(32, 32),
    batch_size=32,
    class_mode='binary')

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [ ]:
# Custom data generator to provide input tensor twice
class DualInputGenerator(tf.keras.utils.Sequence):
    def __init__(self, generator):
        self.generator = generator

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        x, y = self.generator[index]
        return [x, x], y

train_dual_generator = DualInputGenerator(train_generator)
test_dual_generator = DualInputGenerator(test_generator)

# Print the length of the generators to verify
print("Length of train_dual_generator:", len(train_dual_generator))
print("Length of test_dual_generator:", len(test_dual_generator))

# Use the tf.data API for optimized data loading
def create_dataset(generator):
    dataset = tf.data.Dataset.from_generator(
        lambda: generator,
        output_signature=(
            (tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32),
             tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32)),
            tf.TensorSpec(shape=(None,), dtype=tf.float32)
        )
    )
    dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_dual_generator)
test_dataset = create_dataset(test_dual_generator)

# Define callbacks for model checkpoints and early stopping
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        verbose=1,
        restore_best_weights=True)
]


Length of train_dual_generator: 3125
Length of test_dual_generator: 625


In [ ]:
# Use the tf.data API for optimized data loading
def create_dataset(generator):
    dataset = tf.data.Dataset.from_generator(
        lambda: generator,
        output_signature=(
            (tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32),
             tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32)),
            tf.TensorSpec(shape=(None,), dtype=tf.int32)
        )
    )
    dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_dual_generator)
test_dataset = create_dataset(test_dual_generator)

# Define callbacks for model checkpoints and early stopping
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        verbose=1,
        restore_best_weights=True)
]


In [ ]:
# Train the model (only if the lengths are non-zero)
if len(train_dual_generator) > 0 and len(test_dual_generator) > 0:
    model.fit(
        train_dataset,
        steps_per_epoch=len(train_dual_generator),
        epochs=10,
        validation_data=test_dataset,
        validation_steps=len(test_dual_generator),
        callbacks=callbacks)
else:
    print("Error: One or both of the data generators are empty. Check your data paths and ImageDataGenerator setup.")

Epoch 1/10


In [ ]:
# Prediction and Evaluation
test_steps = test_generator.samples // test_generator.batch_size
test_generator.reset()
predictions = model.predict(test_dual_generator, steps=test_steps, verbose=1)

predicted_classes = (predictions > 0.5).astype(int).reshape(-1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()